In [1]:
from numba import cuda
import numpy as np

### Numba Cuda Kernel

This is the easiest way I found to write a Cuda Kernel in Python.
Great talk explaining this: https://www.youtube.com/watch?v=CQDsT81GyS8&t=5402s

Great example: https://github.com/numba/pydata-amsterdam2019-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb

In [2]:
@cuda.jit
def add_kernel(x, y, out):
    tx = cuda.threadIdx.x
    ty = cuda.blockIdx.x
    
    block_size = cuda.blockDim.x
    grid_size = cuda.gridDim.x
    
    start = tx + ty * block_size
    stride = block_size * grid_size
    
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

In [ ]:
n = 100000
x = np.arange(n).astype(np.float32)
y = 2*x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

add_kernel[blocks_per_grid, threads_per_block](x,y,out)
print(out[:10])

In [ ]:
x_device = cuda.to_device(x)
y_device = cuda.to_device(y)
out_device = cuda.device_array_like(x)

### Move the arrays from cpu memory to gpu memory

In [ ]:
%time
add_kernel[blocks_per_grid, threads_per_block](x_device,y_device,out_device)

In [ ]:
%time
x + y